# Tensorflow Mechanics (Practice)

Objective: practice notebook to demo key aspects of the TF library

In [2]:
import tensorflow as tf
import numpy as np

Graph Basics

In [3]:
#A tensor is a data structure similar to an array of d dimensions. 
#We define a graph and then obtain tensor rank and shape:
g = tf.Graph()

with g.as_default():
    #Defining constants - different from placeholders as the data structure has been initialized with values
    t = tf.constant([1,2,3,4])
    #Getting rank of constant
    r = tf.rank(t)
    #Getting shape of constant
    s = t.get_shape()
    
    print('Rank of constant: ', r)
    print('Shape of constant: ', s)

Rank of constant:  Tensor("Rank:0", shape=(), dtype=int32)
Shape of constant:  (4,)


In [5]:
#Notice the rank was not printed because it needs to be executed as part of graph execution:
#We create a session and pass the graph we created as argument:
with tf.Session(graph=g) as sess:
    print("Rank:", r.eval())

Rank: 1


#### The key objective of TF is to create a graph and easily calculate gradients at each node. 
#### The individual steps for building and compiling a graph are:
    1 Instantiate a new and empty graph
    2 Add nodes (tensors and operations) to the graph
    3 Execute the graph:
        a. Start a new session
        b. Initialize the variables in a graph
        c. Run the computation graph in this session

In [6]:
#Recreating the above process:

g = tf.Graph()

with g.as_default():
    a = tf.constant(1, name = 'a')
    b = tf.constant(2, name = 'b')
    c = tf.constant(3, name = 'c')
    
    z = a*b + c    

In [7]:
with tf.Session(graph=g) as sess:
    print('output: ', sess.run(z))

output:  5


In [13]:
#New graph using placeholders:

g = tf.Graph()

with g.as_default():
    #Creating nodes with placeholders
    tf_a = tf.placeholder(tf.int32, shape=[], name = 'tf_a') #Notice the shape is undefined
    tf_b = tf.placeholder(tf.int32, shape=[], name = 'tf_b')
    tf_c = tf.placeholder(tf.int32, shape=[], name = 'tf_c')#This placeholder has rank 3
    #Defining computation nodes
    r1 = tf_a-tf_b
    r2 = 2*r1
    z = r2 + tf_c

In [15]:
with tf.Session(graph=g) as sess:
    #The below dictionary is used to set placeholder values with data arrays
    feed = {tf_a:1, tf_b:2, tf_c:2}
    print('z: ', sess.run(z, feed_dict=feed))

z:  0


In [17]:
#We can also set placeholders for variables of varying size:
g = tf.Graph()

with g.as_default():
    tf_x = tf.placeholder(tf.float32, shape=[None, 2], name='tf_x')
    x_mean = tf.reduce_mean(tf_x, axis=0, name='mean')

In [19]:
np.random.seed(123)
np.set_printoptions(precision=2)

with tf.Session(graph=g) as sess:
    x1 = np.random.uniform(low=0, high=1, size=(5,2)) #size array dim 1 is 5 and dim 2 has to be set to 2
    print('Feeding data with shape: ', x1.shape)
    print('Result: ', sess.run(x_mean, feed_dict={tf_x:x1}))
    
    x2 = np.random.uniform(low=0, high=1, size=(10,2)) #size array dim 1 changed but dim 2 did not
    print('Feeding data with shape: ', x2.shape)
    print('Result: ', sess.run(x_mean, feed_dict={tf_x:x2}))

Feeding data with shape:  (5, 2)
Result:  [0.62 0.47]
Feeding data with shape:  (10, 2)
Result:  [0.46 0.49]


##### Variables: two ways to create them and/or use them:
        1 tf.Variable(<initial-value>, name='variable_name')
        2 tf.get_variable(name, ...)

In [21]:
#Defining variables:
g1 = tf.Graph()

with g1.as_default():
    w = tf.Variable(np.array([[1,2,3,4], [5,6,7,8]]), name='w')
    print(w)

<tf.Variable 'w:0' shape=(2, 4) dtype=int64_ref>


Important: variables are not created in memory until they are initialized with the global_variables_initializer function. Variable values are not set until initialization.

In [22]:
with tf.Session(graph=g1) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

[[1 2 3 4]
 [5 6 7 8]]


Variable Scoping

In [26]:
#Variables can be scoped within a network by means of the variable_scope function:
g = tf.Graph()

with g.as_default():
    
    with tf.variable_scope('net_a'):
        
        with tf.variable_scope('layer-1'):
            w1 = tf.Variable(tf.random_normal(shape=(10,4)), name='weights')
            
        with tf.variable_scope('layer-2'):
            w2 = tf.Variable(tf.random_normal(shape=(20,10)), name='weights')
            
    with tf.variable_scope('net_b'):
        
        with tf.variable_scope('layer-3'):
            w3 = tf.Variable(tf.random_normal(shape=(10,10), name='weights'))

print(w1,'\n',w2,'\n',w3)

<tf.Variable 'net_a/layer-1/weights:0' shape=(10, 4) dtype=float32_ref> 
 <tf.Variable 'net_a/layer-2/weights:0' shape=(20, 10) dtype=float32_ref> 
 <tf.Variable 'net_b/layer-3/Variable:0' shape=(10, 10) dtype=float32_ref>


Creating Network by Reusing Variables

In [28]:
#Helper function to create classifier:

def build_classifier(data, labels, n_classes=2):
    data_shape = data.get_shape().as_list()
    #Notice: using get_variable to create a new variable
    #Also notice the data type is tf.float32 not just float32
    weights = tf.get_variable(name='weights', shape=(data_shape[1], n_classes), dtype=tf.float32) 
    bias = tf.get_variable(name='bias', initializer=tf.zeros(shape=n_classes))
    logits = tf.add(tf.matmul(data, weights), bias, name='logits')
    
    return logits, tf.nn.softmax(logits)

In [33]:
#Helper function to generate data from subnetwork:

def build_generator(data, n_hidden):
    data_shape = data.get_shape().as_list()
    
    w1 = tf.Variable(tf.random_normal(shape=(data_shape[1], n_hidden)), name='w1')
    b1 = tf.Variable(tf.zeros(shape=n_hidden), name='b1')
    hidden = tf.add(tf.matmul(data, w1), b1, name = 'hidden_pre_activation')
    hidden = tf.nn.relu(hidden, 'hidden_activation')
    
    w2 = tf.Variable(tf.random_normal(shape=(n_hidden, data_shape[1])), name='w2')
    b2 = tf.Variable(tf.zeros(shape=data_shape[1]), name='b2')
    output = tf.add(tf.matmul(hidden, w2), b2, name='output')
    
    return output, tf.nn.sigmoid(output)

In [34]:
#Building the Graph:

batch_size = 64
g =  tf.Graph()

with g.as_default():
    tf_X = tf.placeholder(shape=(batch_size,100), dtype = tf.float32, name='tf_X')
    
    #Building the generator for the subgraph:
    with tf.variable_scope('generator'):
        gen_out1 = build_generator(data=tf_X, n_hidden=50)
        
    #Building the classifier:
    with tf.variable_scope('classifier') as scope:
        cls_out1 = build_classifier(data=tf_X, labels = tf.ones(shape=batch_size))
        
        #Reusing the same classifier for generated data:
        scope.reuse_variables()
        cls_out2 = build_classifier(data=gen_out1[1], labels = tf.zeros(shape=batch_size))

In [39]:
with tf.Session(graph=g) as sess:
    #sess.run(tf.global_variables_initializer())
    feed = {tf_X:[3,4]}
    
    print(sess.run(cls_out2, feed_dict=feed))  

ValueError: Cannot feed value of shape (2,) for Tensor 'tf_X:0', which has shape '(64, 100)'